In [1]:
import requests
import pandas as pd

In [2]:
url = 'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=IBM&interval=5min&apikey=XO6PY6AABZ9SDAXI'
r = requests.get(url)
data = r.json()

print(data)

{'Meta Data': {'1. Information': 'Intraday (5min) open, high, low, close prices and volume', '2. Symbol': 'IBM', '3. Last Refreshed': '2025-05-30 19:55:00', '4. Interval': '5min', '5. Output Size': 'Compact', '6. Time Zone': 'US/Eastern'}, 'Time Series (5min)': {'2025-05-30 19:55:00': {'1. open': '259.0500', '2. high': '259.0500', '3. low': '259.0000', '4. close': '259.0000', '5. volume': '31'}, '2025-05-30 19:45:00': {'1. open': '259.0500', '2. high': '259.0500', '3. low': '259.0500', '4. close': '259.0500', '5. volume': '5'}, '2025-05-30 19:30:00': {'1. open': '257.6500', '2. high': '257.6500', '3. low': '257.6500', '4. close': '257.6500', '5. volume': '4'}, '2025-05-30 19:20:00': {'1. open': '259.0000', '2. high': '259.0000', '3. low': '259.0000', '4. close': '259.0000', '5. volume': '50'}, '2025-05-30 19:15:00': {'1. open': '258.8250', '2. high': '259.0000', '3. low': '258.8250', '4. close': '259.0000', '5. volume': '10'}, '2025-05-30 19:00:00': {'1. open': '259.0600', '2. high': '

In [3]:
print(data.keys())

dict_keys(['Meta Data', 'Time Series (5min)'])


In [4]:
df = pd.DataFrame.from_dict(data['Time Series (5min)'], orient='index')
df = df.reset_index().rename(columns={'index': 'Date'})
df['Date'] = pd.to_datetime(df['Date'])

print(df.head())

                 Date   1. open   2. high    3. low  4. close 5. volume
0 2025-05-30 19:55:00  259.0500  259.0500  259.0000  259.0000        31
1 2025-05-30 19:45:00  259.0500  259.0500  259.0500  259.0500         5
2 2025-05-30 19:30:00  257.6500  257.6500  257.6500  257.6500         4
3 2025-05-30 19:20:00  259.0000  259.0000  259.0000  259.0000        50
4 2025-05-30 19:15:00  258.8250  259.0000  258.8250  259.0000        10


In [5]:
print("Number of columns:", len(df.columns))
print("Column names:", df.columns.tolist())

Number of columns: 6
Column names: ['Date', '1. open', '2. high', '3. low', '4. close', '5. volume']


In [6]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows

In [7]:
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)
df.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
df['Open'] = pd.to_numeric(df['Open'], errors='coerce')
df['High'] = pd.to_numeric(df['High'], errors='coerce')
df['Low'] = pd.to_numeric(df['Low'], errors='coerce')
df['Close'] = pd.to_numeric(df['Close'], errors='coerce')
df['Volume'] = pd.to_numeric(df['Volume'], errors='coerce')

In [8]:
df['Daily Change'] = df['Close'] - df['Open']
df['Percentage Change'] = (df['Close'] - df['Open']) / df['Open'] * 100

In [9]:
df.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Daily Change', 'Percentage Change']


In [10]:
print("Number of columns:", len(df.columns))
print("Column names:", df.columns.tolist())

Number of columns: 8
Column names: ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Daily Change', 'Percentage Change']


In [11]:
from sqlalchemy import create_engine

In [12]:
engine = create_engine('sqlite:///stocks_data.db', echo=False)
df.to_sql('stock_prices', con=engine, if_exists='replace', index=False)

100

In [15]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px

app = dash.Dash(__name__)

data = pd.read_sql('SELECT * FROM stock_prices', con=engine)

fig = px.line(data, x='Date', y='Open', title='Stock Closing Prices Over Open')

app.layout = html.Div(children=[
    html.H1(children='Real-Time Stock Dashboard'),
    dcc.Graph(id='line-plot', figure=fig)
])

if __name__ == '__main__':
    app.run(debug=True, port=8052)

In [16]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd

app = dash.Dash(__name__)

data = pd.read_sql('SELECT * FROM stock_prices', con=engine)

fig_line = px.line(
    data, 
    x='Date', 
    y='Close', 
    title='Stock Closing Prices Over Time',
    color_discrete_sequence=['red']  
)

fig_bar = px.line(
    data, 
    x='Date', 
    y='Open', 
    title='Stock Volume Over Time',
    color_discrete_sequence=['blue']
)

combined_fig = go.Figure()

for trace in fig_line.data:
    combined_fig.add_trace(trace)

for trace in fig_bar.data:
    combined_fig.add_trace(trace)

combined_fig.update_layout(title='Open vs Close')

app.layout = html.Div(children=[
    html.H1(children='Real-Time Stock Dashboard'),
    dcc.Graph(id='combined-plot', figure=combined_fig)
])

if __name__ == '__main__':
    app.run(debug=True, port=8051)
